# Veri Analizleri

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
from plotly import express
from plotly import graph_objects
from plotly import io

io.templates.default = "plotly_dark"

In [3]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.tsa.stattools import adfuller, acf, breakvar_heteroskedasticity_test

In [4]:
from statsmodels.graphics import tsaplots
import matplotlib.pyplot as plt

In [5]:
import plotly.graph_objects as go
import statsmodels.api as sm 

In [6]:
country_name = ["France", "Greece",	"Italy", "Portugal", "Spain", "Türkiye"]

In [7]:
df_2 = pd.read_excel("OECD VERİLERİ ELİF BAĞCI TEZ V0_python.xlsx", sheet_name=10, header=1 )
df_2.head()

,Unnamed: 0,France,Greece,Italy,Portugal,Spain,Türkiye,t,interruption,interrupted t
0,2000,16.899,19.871,21.158,21.566,21.958,27.806000,1,0,0
1,2001,17.333,19.075,21.920,22.192,21.949,35.512848,2,0,0
2,2002,17.188,19.949,21.865,22.255,22.650,37.713939,3,0,0
3,2003,17.559,24.572,21.375,22.043,21.418,36.040784,4,0,0
4,2004,17.630,24.716,20.996,22.096,20.677,34.124759,5,0,0


In [8]:
def create_corr_plot(series, plot_pacf=False):
    corr_array = pacf(series.dropna(), alpha=0.05) if plot_pacf else acf(series.dropna(), alpha=0.05)
    lower_y = corr_array[1][:,0] - corr_array[0]
    upper_y = corr_array[1][:,1] - corr_array[0]

    fig = go.Figure()
    [fig.add_scatter(x=(x,x), y=(0,corr_array[0][x]), mode='lines',line_color='#3f3f3f') 
     for x in range(len(corr_array[0]))]
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=corr_array[0], mode='markers', marker_color='#1f77b4',
                   marker_size=12)
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)')
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=lower_y, mode='lines',fillcolor='rgba(32, 146, 230,0.3)',
            fill='tonexty', line_color='rgba(255,255,255,0)')
    fig.update_traces(showlegend=False)
    fig.update_xaxes(range=[-1,13])
    fig.update_yaxes(zerolinecolor='#000000')
    
    title='Partial Autocorrelation (PACF)' if plot_pacf else 'Autocorrelation (ACF)'
    fig.update_layout(title=title)
    fig.show()

In [9]:
for country in country_name:
    print("###########################")
    print(f"{country}'nin Zaman Serisi Analizi")
    print("###########################")

    trendlin = express.scatter(
        df_2,
        x="t",
        y=f"{country}",
        title=f"{country}'s Current expenditure on pharmaceuticals (prescribed and over-the-counter medicines) and other medical non-durables, % of current expenditure on health",
        trendline = "ols"
    )

    trendlin.show()
    linear_model = ols(f"{country} ~ t", data=df_2).fit()
    print(linear_model.summary())
    print(" ", end="\n")
    print("###########################")
    print("Verinin Durağanlık Kontrolü", end="\n\n")
    print("H0: Seri durağan değildir.")
    print("H1: Seri durağandır.", end="\n\n")
    adfullerx = adfuller(df_2[[f"{country}"]])
    print(f'Test Stat = {round(adfullerx[0], 4)}, p-value = {round(adfullerx[1], 4)}')

    if adfullerx[1]<0.05:
        print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Yani seri durağandır.")
    elif adfullerx[1]>0.05:
        print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani seri durağan değildir.", end="\n\n")
    print("###########################")
    print("Hataların Normal Dağılması", end="\n\n")
    residual_values = linear_model.resid

    print("H0: Normal dağılım varsayımı sağlanmaktadır.")
    print("H1: Normal dağılım varsayımı sağlanmamaktadır.", end="\n\n")

    test_stat, pvalue = stats.shapiro(residual_values)
    print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue, 4)}')
    if pvalue<0.05:
        print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar normal dağılmamıştır.")
    elif pvalue>0.05:
        print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar normal dağılmıştır.", end="\n\n")
    hist = express.histogram(residual_values, title="Hataların Dağılımı")
    hist.show()

    print("###########################")
    print("Hataların Rassallığı", end="\n\n")

    print("     Hataların Durağanlığı", end="\n\n")

    adfullery = adfuller(residual_values)
    print(f'Test Stat = {round(adfullery[0], 4)}, p-value = {round(adfullery[1], 4)}')

    if adfullery[1]<0.05:
        print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar durağandır.")
    elif adfullery[1]>0.05:
        print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.", end="\n\n")

    resid_scatter = express.scatter(
        df_2,
        x="t",
        y=residual_values,
        title="Hataların Rassallığı"
    )

    resid_scatter.show()


    print("     Hataların Otokorelasyonu", end="\n\n")

    print("Ljung-Box q istatistiğinin Hipotezleri")
    print("H0: Veriler bağımsız (random) olarak dağılmıştır.")
    print("H1: Veriler bağımsız (random) olarak dağılmamıştır.", end="\n\n")

    acf_df = pd.DataFrame(pd.DataFrame(acf(residual_values, qstat=True)).T)
    acf_df.columns = ["ACF", "Ljung-Box q statistic", "p-value"]
    display(acf_df[1:6])

    create_corr_plot(residual_values)


###########################
France'nin Zaman Serisi Analizi
###########################


                            OLS Regression Results                            
Dep. Variable:                 France   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     229.1
Date:                Thu, 02 Feb 2023   Prob (F-statistic):           4.70e-12
Time:                        12:06:04   Log-Likelihood:                -11.000
No. Observations:                  21   AIC:                             26.00
Df Residuals:                      19   BIC:                             28.09
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     18.1235      0.194     93.246      0.0

###########################
Hataların Rassallığı

     Hataların Durağanlığı

Test Stat = -2.4097, p-value = 0.139
p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.



     Hataların Otokorelasyonu

Ljung-Box q istatistiğinin Hipotezleri
H0: Veriler bağımsız (random) olarak dağılmıştır.
H1: Veriler bağımsız (random) olarak dağılmamıştır.



,ACF,Ljung-Box q statistic,p-value
1,0.572032,10.146402,0.006262
2,0.297110,10.189575,0.017022
3,-0.040111,12.738019,0.012629
4,-0.299494,18.610540,0.002271
5,-0.441061,23.510065,0.000642


...

###########################
Türkiye'nin Zaman Serisi Analizi
###########################


                            OLS Regression Results                            
Dep. Variable:                Türkiye   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                     40.02
Date:                Thu, 02 Feb 2023   Prob (F-statistic):           4.52e-06
Time:                        12:06:04   Log-Likelihood:                -49.512
No. Observations:                  21   AIC:                             103.0
Df Residuals:                      19   BIC:                             105.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.4055      1.216     29.930      0.0

###########################
Hataların Rassallığı

     Hataların Durağanlığı

Test Stat = -2.698, p-value = 0.0744
p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.



     Hataların Otokorelasyonu

Ljung-Box q istatistiğinin Hipotezleri
H0: Veriler bağımsız (random) olarak dağılmıştır.
H1: Veriler bağımsız (random) olarak dağılmamıştır.



,ACF,Ljung-Box q statistic,p-value
1,0.280524,1.994589,0.368876
2,-0.060855,2.211657,0.529656
3,-0.089942,2.223191,0.694786
4,-0.020148,3.112462,0.682652
5,-0.171634,4.080039,0.665846


# Interrupted Time Series for Türkiye

    Yukarıda referans ülkelerin ve Türkiye'nin zaman serisi regresyon analizi sonuçlarına bakılırken, bu kısımdan itibaren Türkiye için interrupted time series analysis yapılmıştır.

In [10]:
X = df_2[["t", "interruption", "interrupted t"]]
y = df_2["Türkiye"]

X = sm.add_constant(X) 

analys= sm.OLS(y, X).fit()

trendlin = express.scatter(
        df_2,
        x="t",
        y= "Türkiye",
        title="Türkiye's Current expenditure on pharmaceuticals (prescribed and over-the-counter medicines) and other medical non-durables, % of current expenditure on health",
    )

trendlin.add_trace(
go.Scatter(
    x = [1,5],
    y = [analys.predict()[0], analys.predict()[4]],
)
 )   

trendlin.add_trace(
go.Scatter(
    x = [5,6],
    y = [analys.predict()[4], analys.predict()[5]]
)

 )

trendlin.add_trace(
go.Scatter(
    x = [6,21],
    y = [analys.predict()[5], analys.predict()[20]]
)

 )  

trendlin.show() 



print(analys.summary())

print("###########################")
print("Hataların Normal Dağılması", end="\n\n")
residual_values = analys.resid
print("H0: Normal dağılım varsayımı sağlanmaktadır.")
print("H1: Normal dağılım varsayımı sağlanmamaktadır.", end="\n\n")
test_stat, pvalue = stats.shapiro(residual_values)
print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue, 4)}')
if pvalue<0.05:
    print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar normal dağılmamıştır.")
elif pvalue>0.05:
     print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar normal dağılmıştır.", end="\n\n")
hist = express.histogram(residual_values, title="Hataların Dağılımı")
hist.show()
print("###########################")
print("Hataların Rassallığı", end="\n\n")
print("     Hataların Durağanlığı", end="\n\n")
adfullery = adfuller(residual_values)
print(f'Test Stat = {round(adfullery[0], 4)}, p-value = {round(adfullery[1], 4)}')
if adfullery[1]<0.05:
    print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar durağandır.")
elif adfullery[1]>0.05:
    print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani hatalar durağan değildir.", end="\n\n")
resid_scatter = express.scatter(
    df_2,
    x="t",
    y=residual_values,
    title="Hataların Rassallığı"
)
resid_scatter.show()
print("     Hataların Otokorelasyonu", end="\n\n")
print("Ljung-Box q istatistiğinin Hipotezleri")
print("H0: Veriler bağımsız (random) olarak dağılmıştır.")
print("H1: Veriler bağımsız (random) olarak dağılmamıştır.", end="\n\n")
acf_df = pd.DataFrame(pd.DataFrame(acf(residual_values, qstat=True)).T)
acf_df.columns = ["ACF", "Ljung-Box q statistic", "p-value"]
display(acf_df[1:6])
create_corr_plot(residual_values)

                            OLS Regression Results                            
Dep. Variable:                Türkiye   R-squared:                       0.776
Model:                            OLS   Adj. R-squared:                  0.737
Method:                 Least Squares   F-statistic:                     19.65
Date:                Thu, 02 Feb 2023   Prob (F-statistic):           9.14e-06
Time:                        12:06:04   Log-Likelihood:                -45.695
No. Observations:                  21   AIC:                             99.39
Df Residuals:                      17   BIC:                             103.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            30.2900      2.485     12.189

###########################
Hataların Rassallığı

     Hataların Durağanlığı

Test Stat = -3.465, p-value = 0.0089
p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Hatalar durağandır.


     Hataların Otokorelasyonu

Ljung-Box q istatistiğinin Hipotezleri
H0: Veriler bağımsız (random) olarak dağılmıştır.
H1: Veriler bağımsız (random) olarak dağılmamıştır.



,ACF,Ljung-Box q statistic,p-value
1,0.233825,2.354744,0.308087
2,-0.201716,2.907278,0.406143
3,-0.143497,3.298410,0.509184
4,0.117331,5.606308,0.346430
5,-0.276500,8.313476,0.216025
